### Setting up colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/My Drive/SML/Project2

/content/drive/My Drive/SML/Project2


In [3]:
import os
os.listdir()

['FEMALE.csv',
 'MALE.csv',
 'MIXED.csv',
 'README.md',
 'SML_Code_1103842_Copy.ipynb',
 'SML_Code_1103842.ipynb']

### Importing modules

In [0]:
!pip install xgboost

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
#from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from statistics import mean
from sklearn.tree import DecisionTreeClassifier

### Reading csv files

In [5]:
male = pd.read_csv('MALE.csv')
female = pd.read_csv('FEMALE.csv')
mixed = pd.read_csv('MIXED.csv')
mixed = mixed[mixed['VR Band of Student']!=0].reset_index(drop=True)
#mixed.drop(mixed.loc[mixed['VR Band of Student']==0].index, inplace = True)
#mixed.drop(mixed[mixed['VR Band of Student']==0].index, inplace = True)
print(male.shape)
print(female.shape)
print(mixed.shape)

cat_columns = ["Year", "VR Band of Student", "Ethnic group of student", "School denomination"]
male = pd.get_dummies(male, prefix_sep="_", columns=cat_columns)
female = pd.get_dummies(female, prefix_sep="_", columns=cat_columns)
mixed = pd.get_dummies(mixed, prefix_sep="_", columns=cat_columns)
print(male.head())
print(female.head())
print(mixed.head())
print(male.shape)
print(female.shape)
print(mixed.shape)

(3654, 7)
(4404, 7)
(7289, 7)
   FSM  VR1 Band  ...  School denomination_2  School denomination_3
0   24        21  ...                      1                      0
1   24        21  ...                      1                      0
2   24        21  ...                      1                      0
3   24        21  ...                      1                      0
4   24        21  ...                      1                      0

[5 rows x 23 columns]
   FSM  VR1 Band  ...  School denomination_2  School denomination_3
0   63        15  ...                      0                      0
1   63        15  ...                      0                      0
2   63        15  ...                      0                      0
3   63        15  ...                      0                      0
4   63        15  ...                      0                      0

[5 rows x 23 columns]
   FSM  VR1 Band  ...  School denomination_2  School denomination_3
0   24        18  ...                   

In [6]:
feat_col = list(mixed.columns)
feat_col.remove('Exam Score')
print(feat_col)
print(len(feat_col))

['FSM', 'VR1 Band', 'Year_1', 'Year_2', 'Year_3', 'VR Band of Student_1', 'VR Band of Student_2', 'VR Band of Student_3', 'Ethnic group of student_1', 'Ethnic group of student_2', 'Ethnic group of student_3', 'Ethnic group of student_4', 'Ethnic group of student_5', 'Ethnic group of student_6', 'Ethnic group of student_7', 'Ethnic group of student_8', 'Ethnic group of student_9', 'Ethnic group of student_10', 'Ethnic group of student_11', 'School denomination_1', 'School denomination_2', 'School denomination_3']
22


### Data partition

In [46]:
shuffled = male.sample(frac=1, random_state = 24).reset_index(drop=True)
male_dev, male_test, male_train = np.array_split(shuffled, [100, 200])
male_test = male_test.reset_index(drop=True)
male_train = male_train.reset_index(drop=True)
male_tgt = male_train.sample(n=100, random_state = 24).reset_index(drop=True)
print(male_train.shape)
male_train_pred = male_train.loc[:, 'Exam Score']
print(male_train_pred.shape)
male_train = male_train.loc[:, feat_col]
print(male_train.shape)
male_dev_pred = male_dev.loc[:, 'Exam Score']
male_dev = male_dev.loc[:, feat_col]
male_test_pred = male_test.loc[:, 'Exam Score']
male_test = male_test.loc[:, feat_col]
male_tgt_pred = male_tgt.loc[:, 'Exam Score']
male_tgt = male_tgt.loc[:, feat_col]
"""
print(type(male_train))
print(male_train.head())
print(male_train.shape)
print(male_dev.shape)
print(male_test.shape)
print(gar.shape)
"""

shuffled = female.sample(frac=1, random_state = 24).reset_index(drop=True)
female_dev, female_test, female_train = np.array_split(shuffled, [100, 200])
female_test = female_test.reset_index(drop=True)
female_train = female_train.reset_index(drop=True)
female_tgt = female_train.sample(n=100, random_state = 24).reset_index(drop=True)
print(female_train.shape)
female_train_pred = female_train.loc[:, 'Exam Score']
print(female_train_pred.shape)
female_train = female_train.loc[:, feat_col]
print(female_train.shape)
female_dev_pred = female_dev.loc[:, 'Exam Score']
female_dev = female_dev.loc[:, feat_col]
female_test_pred = female_test.loc[:, 'Exam Score']
female_test = female_test.loc[:, feat_col]
female_tgt_pred = female_tgt.loc[:, 'Exam Score']
female_tgt = female_tgt.loc[:, feat_col]
"""
print(female_train.head())
print(female_train.shape)
print(female_dev.shape)
print(female_test.shape)
print(gar.shape)
"""

shuffled = mixed.sample(frac=1, random_state = 24).reset_index(drop=True)
mixed_dev, mixed_test, mixed_train = np.array_split(shuffled, [100, 200])
mixed_test = mixed_test.reset_index(drop=True)
mixed_train = mixed_train.reset_index(drop=True)
mixed_tgt = mixed_train.sample(n=100, random_state = 24).reset_index(drop=True)
print(mixed_train.shape)
mixed_train_pred = mixed_train.loc[:, 'Exam Score']
print(mixed_train_pred.shape)
mixed_train = mixed_train.loc[:, feat_col]
print(mixed_train.shape)
mixed_dev_pred = mixed_dev.loc[:, 'Exam Score']
mixed_dev = mixed_dev.loc[:, feat_col]
mixed_test_pred = mixed_test.loc[:, 'Exam Score']
mixed_test = mixed_test.loc[:, feat_col]
mixed_tgt_pred = mixed_tgt.loc[:, 'Exam Score']
mixed_tgt = mixed_tgt.loc[:, feat_col]
"""
print(mixed_train.head())
print(mixed_train.shape)
print(mixed_dev.shape)
print(mixed_test.shape)
"""


(3454, 23)
(3454,)
(3454, 22)
(4204, 23)
(4204,)
(4204, 22)
(7089, 23)
(7089,)
(7089, 22)


'\nprint(mixed_train.head())\nprint(mixed_train.shape)\nprint(mixed_dev.shape)\nprint(mixed_test.shape)\n'

In [0]:
""""
print(male_dev.head())
dev = male_dev.copy().reset_index(drop=True)
print(dev.head())
print(dev.loc[:, 'Year':'School denomination'].head())
print(dev.loc[:, 'Exam Score'].head())
print(len(dev))
"""

'"\nprint(male_dev.head())\ndev = male_dev.copy().reset_index(drop=True)\nprint(dev.head())\nprint(dev.loc[:, \'Year\':\'School denomination\'].head())\nprint(dev.loc[:, \'Exam Score\'].head())\nprint(len(dev))\n'

In [0]:
def highlight_min(s):
    is_min = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_min]

### Task 1.1 (Implement all 6 Baseline methods)

### SRCONLY

In [0]:
#{'activation': 'relu', 'alpha': 0.01, 'batch_size': 128, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 200, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 128, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001, 'max_iter': 1000}
ma_train = pd.concat([female_train, mixed_train], ignore_index=True)
ma_train = ma_train.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev = male_dev.copy().reset_index(drop=True)
ma_test = male_test.copy().reset_index(drop=True)
ma_train_pred = pd.concat([female_train_pred, mixed_train_pred], ignore_index=True)
ma_train_pred = ma_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev_pred = male_dev_pred.copy().reset_index(drop=True)
ma_test_pred = male_test_pred.copy().reset_index(drop=True)

fe_train = pd.concat([male_train, mixed_train], ignore_index=True)
fe_train = fe_train.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev = female_dev.copy().reset_index(drop=True)
fe_test = female_test.copy().reset_index(drop=True)
fe_train_pred = pd.concat([male_train_pred, mixed_train_pred], ignore_index=True)
fe_train_pred = fe_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev_pred = female_dev_pred.copy().reset_index(drop=True)
fe_test_pred = female_test_pred.copy().reset_index(drop=True)

mi_train = pd.concat([male_train, female_train], ignore_index=True)
mi_train = mi_train.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev = mixed_dev.copy().reset_index(drop=True)
mi_test = mixed_test.copy().reset_index(drop=True)
mi_train_pred = pd.concat([male_train_pred, female_train_pred], ignore_index=True)
mi_train_pred = mi_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev_pred = mixed_dev_pred.copy().reset_index(drop=True)
mi_test_pred = mixed_test_pred.copy().reset_index(drop=True)

d = {'male':[ma_train, ma_dev, ma_test, ma_train_pred, ma_dev_pred, ma_test_pred], 'female':[fe_train, fe_dev, fe_test, fe_train_pred, fe_dev_pred, fe_test_pred], 'mixed':[mi_train, mi_dev, mi_test, mi_train_pred, mi_dev_pred, mi_test_pred]}
src_first_dev = []
src_second_dev = []
src_first_test = []
src_second_test = []

for item in d:
  df = d[item]
  X_train = df[0]
  Y_train = df[3]
  X_dev = df[1]
  Y_dev = df[4]
  X_test = df[2]
  Y_test = df[5]
  #scaler = StandardScaler()
  #scaler.fit(X_train)
  #X_train = scaler.transform(X_train)
  #X_dev = scaler.transform(X_dev)
  #X_test = scaler.transform(X_test)
  #parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
  #xgb = XGBClassifier(n_estimators=30, max_depth=10, random_state=24)
  #xgb.fit(X_train, Y_train)
  linr = LinearRegression() #normalize=True
  linr.fit(X_train, Y_train)
  src_first_dev.append(mean_squared_error(Y_dev, linr.predict(X_dev)))
  src_first_test.append(mean_squared_error(Y_test, linr.predict(X_test)))
  
  mlp = MLPRegressor(activation='relu', alpha=0.0001, batch_size=128, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000, random_state=24, verbose=True)
  #parameters = {'hidden_layer_sizes':[(100,), (50,50), (30,30,30), (25,25,25,25)], 'activation':['relu', 'identity'], 'alpha':[0.0001, 0.01, 0.1, 1, 10], 'batch_size':[200, 128, 64, 32], 'max_iter':[1000], 'learning_rate_init':[0.001, 0.01, 0.1]}
  #parameters = {'hidden_layer_sizes':[(100,), (50,50), (30,30,30)], 'alpha':[0.0001, 0.01, 1], 'activation':['relu', 'identity'], 'max_iter':[1000], 'batch_size':[200, 128, 64], 'learning_rate_init':[0.001, 0.01, 0.1]}
  #parameters = {'hidden_layer_sizes':[(100,)], 'alpha':[0.0001], 'activation':['relu'], 'max_iter':[1000], 'batch_size':[200], 'learning_rate_init':[0.001, 0.01]}
  #mse = make_scorer(mean_squared_error, greater_is_better=False) #greater_is_better=False
  #clf = GridSearchCV(mlp, parameters, scoring = 'neg_mean_squared_error', n_jobs = -1)#, verbose=True, scoring = mse)
  mlp.fit(X_train, Y_train)
  #print(clf.best_params_)
  #print(clf.cv_results_)
  #mlp = MLPRegressor(hidden_layer_sizes=(100,), learning_rate_init=0.01, verbose=True, max_iter=1000, random_state=24)
  #mlp.fit(X_train, Y_train)
  src_second_dev.append(mean_squared_error(Y_dev, mlp.predict(X_dev)))
  src_second_test.append(mean_squared_error(Y_test, mlp.predict(X_test)))





Iteration 1, loss = 108.66570139
Iteration 2, loss = 75.15090289
Iteration 3, loss = 71.20661604
Iteration 4, loss = 68.32243477
Iteration 5, loss = 65.50988555
Iteration 6, loss = 62.76317216
Iteration 7, loss = 60.27149179
Iteration 8, loss = 58.24121499
Iteration 9, loss = 56.96747127
Iteration 10, loss = 55.87717995
Iteration 11, loss = 55.59715619
Iteration 12, loss = 54.87448394
Iteration 13, loss = 54.47425668
Iteration 14, loss = 54.10991681
Iteration 15, loss = 53.94279295
Iteration 16, loss = 54.05404891
Iteration 17, loss = 53.73277701
Iteration 18, loss = 53.57075254
Iteration 19, loss = 53.30802557
Iteration 20, loss = 53.28916486
Iteration 21, loss = 53.13240656
Iteration 22, loss = 53.09121649
Iteration 23, loss = 53.24988559
Iteration 24, loss = 52.96685552
Iteration 25, loss = 52.71211885
Iteration 26, loss = 52.82176748
Iteration 27, loss = 52.60537781
Iteration 28, loss = 52.69057856
Iteration 29, loss = 52.50889566
Iteration 30, loss = 52.55507271
Iteration 31, loss

In [0]:
print(src_first_dev)
print(src_first_test)
print(src_second_dev)
print(src_second_test)

[113.27534658892085, 132.89938223552974, 116.86139025375269]
[97.8275385322878, 128.92214442252535, 105.00928474693275]
[116.6041364235833, 154.2215724672641, 121.69415753124206]
[99.07345402152727, 134.57657223542978, 105.74110839186143]


In [0]:
src_test_mse = pd.DataFrame(list(zip(src_first_test, src_second_test)), columns=['SRCONLY_REG','SRCONLY_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
src_test_mse.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,SRCONLY_REG,SRCONLY_NN
0,97.827539,99.073454
1,128.922144,134.576572
2,105.009285,105.741108


### TGTONLY

In [0]:
#{'activation': 'identity', 'alpha': 1, 'batch_size': 64, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 64, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'identity', 'alpha': 0.01, 'batch_size': 64, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.01, 'max_iter': 1000}
#0.001 converged for atleast 1 of the 3 domains, but 0.01 didnt converge for any, 0.1 converged for all domains.
ma_train = pd.concat([male_tgt], ignore_index=True)
ma_train = ma_train.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev = male_dev.copy().reset_index(drop=True)
ma_test = male_test.copy().reset_index(drop=True)
ma_train_pred = pd.concat([male_tgt_pred], ignore_index=True)
ma_train_pred = ma_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev_pred = male_dev_pred.copy().reset_index(drop=True)
ma_test_pred = male_test_pred.copy().reset_index(drop=True)

fe_train = pd.concat([female_tgt], ignore_index=True)
fe_train = fe_train.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev = female_dev.copy().reset_index(drop=True)
fe_test = female_test.copy().reset_index(drop=True)
fe_train_pred = pd.concat([female_tgt_pred], ignore_index=True)
fe_train_pred = fe_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev_pred = female_dev_pred.copy().reset_index(drop=True)
fe_test_pred = female_test_pred.copy().reset_index(drop=True)

mi_train = pd.concat([mixed_tgt], ignore_index=True)
mi_train = mi_train.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev = mixed_dev.copy().reset_index(drop=True)
mi_test = mixed_test.copy().reset_index(drop=True)
mi_train_pred = pd.concat([mixed_tgt_pred], ignore_index=True)
mi_train_pred = mi_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev_pred = mixed_dev_pred.copy().reset_index(drop=True)
mi_test_pred = mixed_test_pred.copy().reset_index(drop=True)

d = {'male':[ma_train, ma_dev, ma_test, ma_train_pred, ma_dev_pred, ma_test_pred], 'female':[fe_train, fe_dev, fe_test, fe_train_pred, fe_dev_pred, fe_test_pred], 'mixed':[mi_train, mi_dev, mi_test, mi_train_pred, mi_dev_pred, mi_test_pred]}

tgt_first_dev = []
tgt_second_dev = []
tgt_first_test = []
tgt_second_test = []

for item in d:
  df = d[item]
  X_train = df[0]
  Y_train = df[3]
  X_dev = df[1]
  Y_dev = df[4]
  X_test = df[2]
  Y_test = df[5]
  #scaler = StandardScaler()
  #scaler.fit(X_train)
  #X_train = scaler.transform(X_train)
  #X_dev = scaler.transform(X_dev)
  #X_test = scaler.transform(X_test)
  #parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
  #xgb = XGBClassifier(n_estimators=30, max_depth=10, random_state=24)
  #xgb.fit(X_train, Y_train)
  linr = LinearRegression() #normalize=True
  linr.fit(X_train, Y_train)
  tgt_first_dev.append(mean_squared_error(Y_dev, linr.predict(X_dev)))
  tgt_first_test.append(mean_squared_error(Y_test, linr.predict(X_test)))

  mlp = MLPRegressor(activation='identity', alpha=0.0001, batch_size=64, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000, random_state=24, verbose=True)
  #mlp = MLPRegressor(random_state=24)
  #parameters = {'hidden_layer_sizes':[(100,), (50,50), (30,30,30), (25,25,25,25)], 'activation':['relu', 'identity'], 'alpha':[0.0001, 0.01, 0.1, 1, 10], 'batch_size':[200, 128, 64, 32], 'max_iter':[1000], 'learning_rate_init':[0.001, 0.01, 0.1]}
  #parameters = {'solver':['adam', 'lbfgs'], 'hidden_layer_sizes':[(100,), (50,50), (30,30,30)], 'alpha':[0.0001, 0.01, 1], 'activation':['relu', 'identity'], 'max_iter':[1000], 'batch_size':[200, 128, 64], 'learning_rate_init':[0.001, 0.01, 0.1]}
  #parameters = {'hidden_layer_sizes':[(100,)], 'alpha':[0.0001], 'activation':['relu'], 'max_iter':[1000], 'batch_size':[200], 'learning_rate_init':[0.001, 0.01]}
  #mse = make_scorer(mean_squared_error, greater_is_better=False) #greater_is_better=False
  #clf = GridSearchCV(mlp, parameters, scoring = 'neg_mean_squared_error', n_jobs = -1)#, verbose=True, scoring = mse)
  mlp.fit(X_train, Y_train)
  #print(clf.best_params_)
  #mlp = MLPRegressor(hidden_layer_sizes=(100,), learning_rate_init=0.1, verbose=True, max_iter=1000, random_state=24)
  #mlp.fit(X_train, Y_train)
  tgt_second_dev.append(mean_squared_error(Y_dev, mlp.predict(X_dev)))
  tgt_second_test.append(mean_squared_error(Y_test, mlp.predict(X_test)))

Iteration 1, loss = 281.79653501
Iteration 2, loss = 233.25873066
Iteration 3, loss = 192.79598434
Iteration 4, loss = 161.57151159
Iteration 5, loss = 138.06078808
Iteration 6, loss = 124.03595368
Iteration 7, loss = 114.26131012
Iteration 8, loss = 111.76166502
Iteration 9, loss = 111.70198090
Iteration 10, loss = 111.54186145
Iteration 11, loss = 112.29727990
Iteration 12, loss = 112.30106095
Iteration 13, loss = 111.06258230
Iteration 14, loss = 108.85379034
Iteration 15, loss = 106.03427221
Iteration 16, loss = 103.12511373
Iteration 17, loss = 100.33988657
Iteration 18, loss = 98.48271229
Iteration 19, loss = 97.12407678
Iteration 20, loss = 95.75886726
Iteration 21, loss = 95.01278341
Iteration 22, loss = 94.24073859
Iteration 23, loss = 93.50866049
Iteration 24, loss = 92.97146925
Iteration 25, loss = 92.04491480
Iteration 26, loss = 91.08551013
Iteration 27, loss = 90.27594862
Iteration 28, loss = 89.33301236
Iteration 29, loss = 88.67290227
Iteration 30, loss = 88.04159671
It

In [0]:
print(tgt_first_dev)
print(tgt_first_test)
print(tgt_second_dev)
print(tgt_second_test)

[178.44935124396977, 144.94778932485468, 119.44926488824116]
[132.91106596637482, 126.74614554665905, 106.81075073833799]
[160.05877365623783, 145.8450266135486, 129.37027559142803]
[121.07140954075066, 130.1072141749327, 102.25809954294813]


In [0]:
tgt_test_mse = pd.DataFrame(list(zip(tgt_first_test, tgt_second_test)), columns=['TGTONLY_REG','TGTONLY_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
tgt_test_mse.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,TGTONLY_REG,TGTONLY_NN
0,132.911066,121.071410
1,126.746146,130.107214
2,106.810751,102.258100


### ALL

In [0]:
#{'activation': 'relu', 'alpha': 1, 'batch_size': 64, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 64, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 128, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
ma_train = pd.concat([female_train, mixed_train, male_tgt], ignore_index=True)
ma_train = ma_train.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev = male_dev.copy().reset_index(drop=True)
ma_test = male_test.copy().reset_index(drop=True)
ma_train_pred = pd.concat([female_train_pred, mixed_train_pred, male_tgt_pred], ignore_index=True)
ma_train_pred = ma_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev_pred = male_dev_pred.copy().reset_index(drop=True)
ma_test_pred = male_test_pred.copy().reset_index(drop=True)

fe_train = pd.concat([male_train, mixed_train, female_tgt], ignore_index=True)
fe_train = fe_train.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev = female_dev.copy().reset_index(drop=True)
fe_test = female_test.copy().reset_index(drop=True)
fe_train_pred = pd.concat([male_train_pred, mixed_train_pred, female_tgt_pred], ignore_index=True)
fe_train_pred = fe_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev_pred = female_dev_pred.copy().reset_index(drop=True)
fe_test_pred = female_test_pred.copy().reset_index(drop=True)

mi_train = pd.concat([male_train, female_train, mixed_tgt], ignore_index=True)
mi_train = mi_train.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev = mixed_dev.copy().reset_index(drop=True)
mi_test = mixed_test.copy().reset_index(drop=True)
mi_train_pred = pd.concat([male_train_pred, female_train_pred, mixed_tgt_pred], ignore_index=True)
mi_train_pred = mi_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev_pred = mixed_dev_pred.copy().reset_index(drop=True)
mi_test_pred = mixed_test_pred.copy().reset_index(drop=True)

d = {'male':[ma_train, ma_dev, ma_test, ma_train_pred, ma_dev_pred, ma_test_pred], 'female':[fe_train, fe_dev, fe_test, fe_train_pred, fe_dev_pred, fe_test_pred], 'mixed':[mi_train, mi_dev, mi_test, mi_train_pred, mi_dev_pred, mi_test_pred]}

all_first_dev = []
all_second_dev = []
all_first_test = []
all_second_test = []

for item in d:
  df = d[item]
  X_train = df[0]
  Y_train = df[3]
  X_dev = df[1]
  Y_dev = df[4]
  X_test = df[2]
  Y_test = df[5]
  #scaler = StandardScaler()
  #scaler.fit(X_train)
  #X_train = scaler.transform(X_train)
  #X_dev = scaler.transform(X_dev)
  #X_test = scaler.transform(X_test)
  #parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
  #xgb = XGBClassifier(n_estimators=30, max_depth=10, random_state=24)
  #xgb.fit(X_train, Y_train)
  linr = LinearRegression() #normalize=True
  linr.fit(X_train, Y_train)
  all_first_dev.append(mean_squared_error(Y_dev, linr.predict(X_dev)))
  all_first_test.append(mean_squared_error(Y_test, linr.predict(X_test)))
  
  mlp = MLPRegressor(activation='relu', alpha=0.0001, batch_size=64, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000, random_state=24, verbose=True)
  mlp.fit(X_train, Y_train)
  #mlp = MLPRegressor(random_state=24)
  #parameters = {'hidden_layer_sizes':[(100,), (50,50), (30,30,30), (25,25,25,25)], 'activation':['relu', 'identity'], 'alpha':[0.0001, 0.01, 0.1, 1, 10], 'batch_size':[200, 128, 64, 32], 'max_iter':[1000], 'learning_rate_init':[0.001, 0.01, 0.1]}
  #parameters = {'hidden_layer_sizes':[(100,), (50,50), (30,30,30)], 'alpha':[0.0001, 0.01, 1], 'activation':['relu', 'identity'], 'max_iter':[1000], 'batch_size':[200, 128, 64], 'learning_rate_init':[0.001, 0.01, 0.1]}
  #parameters = {'hidden_layer_sizes':[(100,)], 'alpha':[0.0001], 'activation':['relu'], 'max_iter':[1000], 'batch_size':[200], 'learning_rate_init':[0.001, 0.01]}
  #mse = make_scorer(mean_squared_error, greater_is_better=False) #greater_is_better=False
  #clf = GridSearchCV(mlp, parameters, scoring = 'neg_mean_squared_error', n_jobs = -1)#, verbose=True, scoring = mse)
  #clf.fit(X_train, Y_train)
  #print(clf.best_params_)
  all_second_dev.append(mean_squared_error(Y_dev, mlp.predict(X_dev)))
  all_second_test.append(mean_squared_error(Y_test, mlp.predict(X_test)))

Iteration 1, loss = 90.57087376
Iteration 2, loss = 70.66524092
Iteration 3, loss = 65.66969275
Iteration 4, loss = 61.48277766
Iteration 5, loss = 58.26308728
Iteration 6, loss = 56.78273483
Iteration 7, loss = 55.61869881
Iteration 8, loss = 55.09261635
Iteration 9, loss = 55.13816783
Iteration 10, loss = 54.66204726
Iteration 11, loss = 54.46584194
Iteration 12, loss = 54.12426807
Iteration 13, loss = 53.96487082
Iteration 14, loss = 53.59281066
Iteration 15, loss = 53.55545876
Iteration 16, loss = 53.27520410
Iteration 17, loss = 53.33190693
Iteration 18, loss = 53.16687480
Iteration 19, loss = 53.45027587
Iteration 20, loss = 53.76713174
Iteration 21, loss = 52.75089942
Iteration 22, loss = 52.67303329
Iteration 23, loss = 52.71645600
Iteration 24, loss = 52.78494090
Iteration 25, loss = 54.01756460
Iteration 26, loss = 52.53711401
Iteration 27, loss = 52.40098890
Iteration 28, loss = 52.60755068
Iteration 29, loss = 52.88218699
Iteration 30, loss = 52.49398524
Iteration 31, loss 

In [0]:
print(all_first_dev)
print(all_first_test)
print(all_second_dev)
print(all_second_test)

[113.4607742265285, 132.54585991326113, 116.45332237053626]
[97.50095830199635, 128.5449591098929, 104.76716171617446]
[111.97982092426429, 137.9519159736161, 121.9613100137307]
[92.23586416486971, 125.68273536311342, 101.46935231677621]


In [0]:
all_test_mse = pd.DataFrame(list(zip(all_first_test, all_second_test)), columns=['ALL_REG','ALL_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
all_test_mse.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,ALL_REG,ALL_NN
0,97.500958,92.235864
1,128.544959,125.682735
2,104.767162,101.469352


### WEIGHTED

In [0]:
#{'activation': 'relu', 'alpha': 0.01, 'batch_size': 64, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 64, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.01, 'batch_size': 64, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
ma_inter = pd.concat([female_train, mixed_train], ignore_index=True)
n = len(ma_inter)//len(male_tgt)
tgt_inter = pd.concat([male_tgt]*n, ignore_index=True)
ma_train = pd.concat([ma_inter, tgt_inter], ignore_index=True)
#print(ma_inter.shape, n, tgt_inter.shape, ma_train.shape)
ma_train = ma_train.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev = male_dev.copy().reset_index(drop=True)
ma_test = male_test.copy().reset_index(drop=True)
ma_inter_pred = pd.concat([female_train_pred, mixed_train_pred], ignore_index=True)
n = len(ma_inter_pred)//len(male_tgt_pred)
tgt_inter_pred = pd.concat([male_tgt_pred]*n, ignore_index=True)
ma_train_pred = pd.concat([ma_inter_pred, tgt_inter_pred], ignore_index=True)
ma_train_pred = ma_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev_pred = male_dev_pred.copy().reset_index(drop=True)
ma_test_pred = male_test_pred.copy().reset_index(drop=True)

fe_inter = pd.concat([male_train, mixed_train], ignore_index=True)
n = len(fe_inter)//len(female_tgt)
tgt_inter = pd.concat([female_tgt]*n, ignore_index=True)
fe_train = pd.concat([fe_inter, tgt_inter], ignore_index=True)
fe_train = fe_train.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev = female_dev.copy().reset_index(drop=True)
fe_test = female_test.copy().reset_index(drop=True)
fe_inter_pred = pd.concat([male_train_pred, mixed_train_pred], ignore_index=True)
n = len(fe_inter_pred)//len(female_tgt_pred)
tgt_inter_pred = pd.concat([female_tgt_pred]*n, ignore_index=True)
fe_train_pred = pd.concat([fe_inter_pred, tgt_inter_pred], ignore_index=True)
fe_train_pred = fe_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev_pred = female_dev_pred.copy().reset_index(drop=True)
fe_test_pred = female_test_pred.copy().reset_index(drop=True)

mi_inter = pd.concat([male_train, female_train], ignore_index=True)
n = len(mi_inter)//len(mixed_tgt)
tgt_inter = pd.concat([mixed_tgt]*n, ignore_index=True)
mi_train = pd.concat([mi_inter, tgt_inter], ignore_index=True)
mi_train = mi_train.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev = mixed_dev.copy().reset_index(drop=True)
mi_test = mixed_test.copy().reset_index(drop=True)
mi_inter_pred = pd.concat([male_train_pred, female_train_pred], ignore_index=True)
n = len(mi_inter_pred)//len(mixed_tgt_pred)
tgt_inter_pred = pd.concat([mixed_tgt_pred]*n, ignore_index=True)
mi_train_pred = pd.concat([mi_inter_pred, tgt_inter_pred], ignore_index=True)
mi_train_pred = mi_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev_pred = mixed_dev_pred.copy().reset_index(drop=True)
mi_test_pred = mixed_test_pred.copy().reset_index(drop=True)

d = {'male':[ma_train, ma_dev, ma_test, ma_train_pred, ma_dev_pred, ma_test_pred], 'female':[fe_train, fe_dev, fe_test, fe_train_pred, fe_dev_pred, fe_test_pred], 'mixed':[mi_train, mi_dev, mi_test, mi_train_pred, mi_dev_pred, mi_test_pred]}

wei_first_dev = []
wei_second_dev = []
wei_first_test = []
wei_second_test = []

for item in d:
  df = d[item]
  X_train = df[0]
  Y_train = df[3]
  X_dev = df[1]
  Y_dev = df[4]
  X_test = df[2]
  Y_test = df[5]
  #scaler = StandardScaler()
  #scaler.fit(X_train)
  #X_train = scaler.transform(X_train)
  #X_dev = scaler.transform(X_dev)
  #X_test = scaler.transform(X_test)
  #parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
  #xgb = XGBClassifier(n_estimators=30, max_depth=10, random_state=24)
  #xgb.fit(X_train, Y_train)
  linr = LinearRegression() #normalize=True
  linr.fit(X_train, Y_train)
  wei_first_dev.append(mean_squared_error(Y_dev, linr.predict(X_dev)))
  wei_first_test.append(mean_squared_error(Y_test, linr.predict(X_test)))
  
  mlp = MLPRegressor(activation='relu', alpha=0.01, batch_size=64, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000, random_state=24, verbose=True)
  mlp.fit(X_train, Y_train)
  #print("Before GridSearch")
  #mlp = MLPRegressor(random_state=24)
  #parameters = {'hidden_layer_sizes':[(100,), (50,50), (30,30,30), (25,25,25,25)], 'activation':['relu', 'identity'], 'alpha':[0.0001, 0.01, 0.1, 1, 10], 'batch_size':[200, 128, 64, 32], 'max_iter':[1000], 'learning_rate_init':[0.001, 0.01, 0.1]}
  #parameters = {'hidden_layer_sizes':[(100,)], 'alpha':[0.0001, 0.01, 1], 'activation':['relu', 'identity'], 'max_iter':[1000], 'batch_size':[200, 128, 64], 'learning_rate_init':[0.001]}
  #parameters = {'hidden_layer_sizes':[(100,)], 'alpha':[0.0001], 'activation':['relu'], 'max_iter':[1000], 'batch_size':[200], 'learning_rate_init':[0.001]}
  #mse = make_scorer(mean_squared_error, greater_is_better=False) #greater_is_better=False
  #clf = GridSearchCV(mlp, parameters, scoring = 'neg_mean_squared_error', n_jobs = -1)#, verbose=True, scoring = mse)
  #clf.fit(X_train, Y_train)
  #print("After GridSearch")
  #print(clf.best_params_)
  wei_second_dev.append(mean_squared_error(Y_dev, mlp.predict(X_dev)))
  wei_second_test.append(mean_squared_error(Y_test, mlp.predict(X_test)))


Iteration 1, loss = 84.91151425
Iteration 2, loss = 67.98788826
Iteration 3, loss = 61.52862150
Iteration 4, loss = 59.11651456
Iteration 5, loss = 57.93287856
Iteration 6, loss = 56.99084990
Iteration 7, loss = 56.36272827
Iteration 8, loss = 55.56938252
Iteration 9, loss = 54.97020709
Iteration 10, loss = 54.35500019
Iteration 11, loss = 53.80221282
Iteration 12, loss = 53.27842850
Iteration 13, loss = 52.73492989
Iteration 14, loss = 52.41009871
Iteration 15, loss = 51.99277815
Iteration 16, loss = 51.43602676
Iteration 17, loss = 51.09914932
Iteration 18, loss = 50.61947813
Iteration 19, loss = 50.33278382
Iteration 20, loss = 49.89320399
Iteration 21, loss = 49.50259653
Iteration 22, loss = 49.23195131
Iteration 23, loss = 48.79916420
Iteration 24, loss = 48.46803616
Iteration 25, loss = 48.25448498
Iteration 26, loss = 48.04121646
Iteration 27, loss = 47.77378615
Iteration 28, loss = 47.28182978
Iteration 29, loss = 47.11430586
Iteration 30, loss = 46.78483848
Iteration 31, loss 

In [0]:
print(wei_first_dev)
print(wei_first_test)
print(wei_second_dev)
print(wei_second_test)

[133.43105170284676, 125.80799632370217, 108.57340989780583]
[101.3566533976719, 117.56137299949785, 101.03580286297819]
[126.38121705032674, 127.72600649339834, 158.57057756038293]
[107.25731990044734, 121.05456791881197, 123.46315811651259]


In [0]:
wei_test_mse = pd.DataFrame(list(zip(wei_first_test, wei_second_test)), columns=['WEIGHTED_REG','WEIGHTED_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
wei_test_mse.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,WEIGHTED_REG,WEIGHTED_NN
0,101.356653,107.257320
1,117.561373,121.054568
2,101.035803,123.463158


### PRED

In [0]:
#{'activation': 'identity', 'alpha': 1, 'batch_size': 64, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 1, 'batch_size': 64, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 1, 'batch_size': 200, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.1, 'max_iter': 1000}

ma_train = pd.concat([female_train, mixed_train], ignore_index=True)
ma_train = ma_train.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev = male_dev.copy().reset_index(drop=True)
ma_test = male_test.copy().reset_index(drop=True)
ma_train_pred = pd.concat([female_train_pred, mixed_train_pred], ignore_index=True)
ma_train_pred = ma_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev_pred = male_dev_pred.copy().reset_index(drop=True)
ma_test_pred = male_test_pred.copy().reset_index(drop=True)
ma_tgt = male_tgt.copy().reset_index(drop=True)
ma_tgt_pred = male_tgt_pred.copy().reset_index(drop=True)

fe_train = pd.concat([male_train, mixed_train], ignore_index=True)
fe_train = fe_train.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev = female_dev.copy().reset_index(drop=True)
fe_test = female_test.copy().reset_index(drop=True)
fe_train_pred = pd.concat([male_train_pred, mixed_train_pred], ignore_index=True)
fe_train_pred = fe_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev_pred = female_dev_pred.copy().reset_index(drop=True)
fe_test_pred = female_test_pred.copy().reset_index(drop=True)
fe_tgt = female_tgt.copy().reset_index(drop=True)
fe_tgt_pred = female_tgt_pred.copy().reset_index(drop=True)

mi_train = pd.concat([male_train, female_train], ignore_index=True)
mi_train = mi_train.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev = mixed_dev.copy().reset_index(drop=True)
mi_test = mixed_test.copy().reset_index(drop=True)
mi_train_pred = pd.concat([male_train_pred, female_train_pred], ignore_index=True)
mi_train_pred = mi_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev_pred = mixed_dev_pred.copy().reset_index(drop=True)
mi_test_pred = mixed_test_pred.copy().reset_index(drop=True)
mi_tgt = mixed_tgt.copy().reset_index(drop=True)
mi_tgt_pred = mixed_tgt_pred.copy().reset_index(drop=True)

#d = {'male':[ma_train, ma_dev, ma_test, male_tgt], 'female':[fe_train, fe_dev, fe_test, female_tgt], 'mixed':[mi_train, mi_dev, mi_test, mixed_tgt]}
d = {'male':[ma_train, ma_dev, ma_test, ma_train_pred, ma_dev_pred, ma_test_pred, ma_tgt, ma_tgt_pred], 'female':[fe_train, fe_dev, fe_test, fe_train_pred, fe_dev_pred, fe_test_pred, fe_tgt, fe_tgt_pred], 'mixed':[mi_train, mi_dev, mi_test, mi_train_pred, mi_dev_pred, mi_test_pred, mi_tgt, mi_tgt_pred]}

pre_first_dev = []
pre_second_dev = []
pre_first_test = []
pre_second_test = []

for item in d:
  df = d[item]
  X_train = df[0]
  Y_train = df[3]
  X_dev = df[1]
  Y_dev = df[4]
  X_test = df[2]
  Y_test = df[5]
  X_tgt = df[6]
  Y_tgt = df[7]
  #scaler = StandardScaler()
  #scaler.fit(X_train)
  #X_train = scaler.transform(X_train)
  #print(type(X_train))
  #X_dev = scaler.transform(X_dev)
  #X_test = scaler.transform(X_test)
  #parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
  #xgb = XGBClassifier(n_estimators=30, max_depth=10, random_state=24)
  #xgb.fit(X_train, Y_train)

  #Training and running the SRCONLY model first
  src_linr = LinearRegression() #normalize=True
  src_linr.fit(X_train, Y_train)
  #X_tgt['Pred'] = src_linr.predict(X_tgt)
  Y_hat_tgt = src_linr.predict(X_tgt)
  X_tgt_new = X_tgt.copy()
  X_tgt_new['Pred'] = Y_hat_tgt
  #X_tgt_new = X_tgt_new.sample(n=100, random_state = 24).reset_index(drop=True)
  #X_dev['Pred'] = src_linr.predict(X_dev)
  Y_hat_dev = src_linr.predict(X_dev)
  X_dev_new = X_dev.copy()
  X_dev_new['Pred'] = Y_hat_dev
  #X_dev_new = X_dev_new.sample(n=100, random_state = 24).reset_index(drop=True)
  #X_test['Pred'] = src_linr.predict(X_test)
  Y_hat_test = src_linr.predict(X_test)
  X_test_new = X_test.copy()
  X_test_new['Pred'] = Y_hat_test
  #X_test_new = X_test_new.sample(n=100, random_state = 24).reset_index(drop=True)
  tgt_linr = LinearRegression() #normalize=True
  tgt_linr.fit(X_tgt_new, Y_tgt)
  pre_first_dev.append(mean_squared_error(Y_dev, tgt_linr.predict(X_dev_new)))
  pre_first_test.append(mean_squared_error(Y_test, tgt_linr.predict(X_test_new)))

  #Training and running the TGTONLY model now
  src_mlp = MLPRegressor(activation='relu', alpha=0.0001, batch_size=128, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000, random_state=24, verbose=True)
  src_mlp.fit(X_train, Y_train)
  #X_tgt['Pred'] = src_linr.predict(X_tgt)
  Y_hat_tgt = src_mlp.predict(X_tgt)
  X_tgt_new = X_tgt.copy()
  X_tgt_new['Pred'] = Y_hat_tgt
  #X_tgt_new = X_tgt_new.sample(n=100, random_state = 24).reset_index(drop=True)
  #X_dev['Pred'] = src_linr.predict(X_dev)
  Y_hat_dev = src_mlp.predict(X_dev)
  X_dev_new = X_dev.copy()
  X_dev_new['Pred'] = Y_hat_dev
  #X_dev_new = X_dev_new.sample(n=100, random_state = 24).reset_index(drop=True)
  #X_test['Pred'] = src_linr.predict(X_test)
  Y_hat_test = src_mlp.predict(X_test)
  X_test_new = X_test.copy()
  X_test_new['Pred'] = Y_hat_test
  #X_test_new = X_test_new.sample(n=100, random_state = 24).reset_index(drop=True)
  tgt_mlp = MLPRegressor(activation='relu', alpha=1, batch_size=64, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000, random_state=24, verbose=True)
  tgt_mlp.fit(X_tgt_new, Y_tgt)
  #tgt_mlp = MLPRegressor(random_state=24)
  #parameters = {'hidden_layer_sizes':[(100,), (50,50), (30,30,30), (25,25,25,25)], 'activation':['relu', 'identity'], 'alpha':[0.0001, 0.01, 0.1, 1, 10], 'batch_size':[200, 128, 64, 32], 'max_iter':[1000], 'learning_rate_init':[0.001, 0.01, 0.1]}
  #parameters = {'hidden_layer_sizes':[(100,), (50,50), (30,30,30)], 'alpha':[0.0001, 0.01, 1], 'activation':['relu', 'identity'], 'max_iter':[1000], 'batch_size':[200, 128, 64], 'learning_rate_init':[0.001, 0.01, 0.1]}
  #parameters = {'hidden_layer_sizes':[(100,)], 'alpha':[0.0001], 'activation':['relu'], 'max_iter':[1000], 'batch_size':[200], 'learning_rate_init':[0.001, 0.01]}
  #mse = make_scorer(mean_squared_error, greater_is_better=False) #greater_is_better=False
  #clf = GridSearchCV(tgt_mlp, parameters, scoring = 'neg_mean_squared_error', n_jobs = -1)#, verbose=True, scoring = mse)
  #clf.fit(X_tgt_new, Y_tgt)
  #print(clf.best_params_)

  pre_second_dev.append(mean_squared_error(Y_dev, tgt_mlp.predict(X_dev_new)))
  pre_second_test.append(mean_squared_error(Y_test, tgt_mlp.predict(X_test_new)))

Iteration 1, loss = 108.66570139
Iteration 2, loss = 75.15090289
Iteration 3, loss = 71.20661604
Iteration 4, loss = 68.32243477
Iteration 5, loss = 65.50988555
Iteration 6, loss = 62.76317216
Iteration 7, loss = 60.27149179
Iteration 8, loss = 58.24121499
Iteration 9, loss = 56.96747127
Iteration 10, loss = 55.87717995
Iteration 11, loss = 55.59715619
Iteration 12, loss = 54.87448394
Iteration 13, loss = 54.47425668
Iteration 14, loss = 54.10991681
Iteration 15, loss = 53.94279295
Iteration 16, loss = 54.05404891
Iteration 17, loss = 53.73277701
Iteration 18, loss = 53.57075254
Iteration 19, loss = 53.30802557
Iteration 20, loss = 53.28916486
Iteration 21, loss = 53.13240656
Iteration 22, loss = 53.09121649
Iteration 23, loss = 53.24988559
Iteration 24, loss = 52.96685552
Iteration 25, loss = 52.71211885
Iteration 26, loss = 52.82176748
Iteration 27, loss = 52.60537781
Iteration 28, loss = 52.69057856
Iteration 29, loss = 52.50889566
Iteration 30, loss = 52.55507271
Iteration 31, loss

In [0]:
print(pre_first_dev)
print(pre_first_test)
print(pre_second_dev)
print(pre_second_test)

[178.44935124396926, 145.1573212329774, 118.81744496060324]
[132.91106596637502, 126.49427581568047, 106.81075073833799]
[158.81831618574105, 167.59368397359776, 151.33936917910464]
[125.09476674347545, 133.02428005637884, 109.60629978420262]


In [0]:
pre_test_mse = pd.DataFrame(list(zip(pre_first_test, pre_second_test)), columns=['PRED_REG','PRED_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
pre_test_mse.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,PRED_REG,PRED_NN
0,132.911066,125.094767
1,126.494276,133.024280
2,106.810751,109.606300


### LININT

In [0]:
ma_train = pd.concat([female_train, mixed_train], ignore_index=True)
ma_train = ma_train.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev = male_dev.copy().reset_index(drop=True)
ma_test = male_test.copy().reset_index(drop=True)
ma_train_pred = pd.concat([female_train_pred, mixed_train_pred], ignore_index=True)
ma_train_pred = ma_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev_pred = male_dev_pred.copy().reset_index(drop=True)
ma_test_pred = male_test_pred.copy().reset_index(drop=True)
ma_tgt = male_tgt.copy().reset_index(drop=True)
ma_tgt_pred = male_tgt_pred.copy().reset_index(drop=True)

fe_train = pd.concat([male_train, mixed_train], ignore_index=True)
fe_train = fe_train.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev = female_dev.copy().reset_index(drop=True)
fe_test = female_test.copy().reset_index(drop=True)
fe_train_pred = pd.concat([male_train_pred, mixed_train_pred], ignore_index=True)
fe_train_pred = fe_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev_pred = female_dev_pred.copy().reset_index(drop=True)
fe_test_pred = female_test_pred.copy().reset_index(drop=True)
fe_tgt = female_tgt.copy().reset_index(drop=True)
fe_tgt_pred = female_tgt_pred.copy().reset_index(drop=True)

mi_train = pd.concat([male_train, female_train], ignore_index=True)
mi_train = mi_train.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev = mixed_dev.copy().reset_index(drop=True)
mi_test = mixed_test.copy().reset_index(drop=True)
mi_train_pred = pd.concat([male_train_pred, female_train_pred], ignore_index=True)
mi_train_pred = mi_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev_pred = mixed_dev_pred.copy().reset_index(drop=True)
mi_test_pred = mixed_test_pred.copy().reset_index(drop=True)
mi_tgt = mixed_tgt.copy().reset_index(drop=True)
mi_tgt_pred = mixed_tgt_pred.copy().reset_index(drop=True)

d = {'male':[ma_train, ma_dev, ma_test, ma_train_pred, ma_dev_pred, ma_test_pred, ma_tgt, ma_tgt_pred], 'female':[fe_train, fe_dev, fe_test, fe_train_pred, fe_dev_pred, fe_test_pred, fe_tgt, fe_tgt_pred], 'mixed':[mi_train, mi_dev, mi_test, mi_train_pred, mi_dev_pred, mi_test_pred, mi_tgt, mi_tgt_pred]}

lin_first_dev = []
lin_second_dev = []
lin_first_test = []
lin_second_test = []

for item in d:
  df = d[item]
  X_train = df[0]
  Y_train = df[3]
  X_dev = df[1]
  Y_dev = df[4]
  X_test = df[2]
  Y_test = df[5]
  X_tgt = df[6]
  Y_tgt = df[7]
  #scaler = StandardScaler()
  #scaler.fit(X_train)
  #X_train = scaler.transform(X_train)
  #print(type(X_train))
  #X_dev = scaler.transform(X_dev)
  #X_test = scaler.transform(X_test)
  #parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
  #xgb = XGBClassifier(n_estimators=30, max_depth=10, random_state=24)
  #xgb.fit(X_train, Y_train)

  src_linr = LinearRegression() #normalize=True
  src_linr.fit(X_train, Y_train)
  tgt_linr = LinearRegression() #normalize=True
  tgt_linr.fit(X_tgt, Y_tgt)
  #print(type(src_linr.predict(X_dev)), type(tgt_linr.predict(X_dev)))
  Y1 = src_linr.predict(X_dev)
  Y2 = tgt_linr.predict(X_dev)
  Y_diff = Y2-Y1
  #w = interpolation(Y_dev, Y2, Y_diff)
  #print(w)
  X = pd.DataFrame(list(zip(Y1, Y_diff)), columns=['Y1', 'Y2-Y1'])
  Y = Y_dev.copy().reset_index(drop=True)
  reg = LinearRegression().fit(X, Y)
  print(reg.coef_)
  Y1_t = src_linr.predict(X_test)
  Y2_t = tgt_linr.predict(X_test)
  Y_diff_t = Y2_t - Y1_t
  #print(Y_diff_t)
  #print(w*Y_diff_t)
  X_t = pd.DataFrame(list(zip(Y1_t, Y_diff_t)), columns=['Y1', 'Y2-Y1'])
  Y_t = reg.predict(X_t)
  lin_first_test.append(mean_squared_error(Y_test, Y_t))
  #print(mean_squared_error(Y_test, Y_t))

  src_mlp = MLPRegressor(activation='relu', alpha=0.0001, batch_size=128, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000, random_state=24)
  src_mlp.fit(X_train, Y_train)
  tgt_mlp = MLPRegressor(activation='identity', alpha=0.0001, batch_size=64, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000, random_state=24)
  tgt_mlp.fit(X_tgt, Y_tgt)
  #print(type(src_linr.predict(X_dev)), type(tgt_linr.predict(X_dev)))
  Y1 = src_mlp.predict(X_dev)
  Y2 = tgt_mlp.predict(X_dev)
  #print(Y1, Y2)
  Y_diff = Y2 - Y1
  #print(Y_diff)
  X = pd.DataFrame(list(zip(Y1, Y_diff)), columns=['Y1', 'Y2-Y1'])
  Y = Y_dev.copy().reset_index(drop=True)
  reg = LinearRegression().fit(X, Y)
  print(reg.coef_)
  Y1_t = src_mlp.predict(X_test)
  Y2_t = tgt_mlp.predict(X_test)
  Y_diff_t = Y2_t - Y1_t
  #print(Y_diff_t)
  #print(w*Y_diff_t)
  X_t = pd.DataFrame(list(zip(Y1_t, Y_diff_t)), columns=['Y1', 'Y2-Y1'])
  Y_t = reg.predict(X_t)
  lin_second_test.append(mean_squared_error(Y_test, Y_t))
  #print(mean_squared_error(Y_test, Y_hat_t))

[ 1.00253779 -0.2183974 ]
[ 0.98147893 -0.08916908]
[ 1.1055234  -0.23520784]
[ 1.05694119 -0.26456162]
[1.02318455 0.41640874]
[1.05738549 0.42004257]


In [0]:
print(lin_first_test)
print(lin_second_test)

[98.29361651100606, 128.5344633309243, 100.09035938460765]
[91.39444921876091, 124.0185311717055, 102.11794090647473]


In [0]:
lin_test_mse = pd.DataFrame(list(zip(lin_first_test, lin_second_test)), columns=['LININT_REG','LININT_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
lin_test_mse.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,LININT_REG,LININT_NN
0,98.293617,91.394449
1,128.534463,124.018531
2,100.090359,102.117941


### Dev and Test MSE

In [0]:
baseline_dev_mse = pd.DataFrame(list(zip(src_first_dev, src_second_dev, tgt_first_dev, tgt_second_dev, all_first_dev, all_second_dev, 
                                         wei_first_dev, wei_second_dev, pre_first_dev, pre_second_dev)), columns=['SRCONLY_REG', 
                                                                                                                  'SRCONLY_NN', 'TGTONLY REG',
                                                                                                                  'TGTONLY NN', 'ALL_REG',
                                                                                                                  'ALL_NN', 'WEIGHTED_REG',
                                                                                                                  'WEIGHTED_NN', 'PRED_REG',
                                                                                                                  'PRED_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
baseline_dev_mse.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())


,SRCONLY_REG,SRCONLY_NN,TGTONLY REG,TGTONLY NN,ALL_REG,ALL_NN,WEIGHTED_REG,WEIGHTED_NN,PRED_REG,PRED_NN
0,113.275347,116.604136,178.449351,160.058774,113.460774,111.979821,133.431052,126.381217,178.449351,158.818316
1,132.899382,154.221572,144.947789,145.845027,132.545860,137.951916,125.807996,127.726006,145.157321,167.593684
2,116.861390,121.694158,119.449265,129.370276,116.453322,121.961310,108.573410,158.570578,118.817445,151.339369


In [0]:
baseline_test_mse = pd.DataFrame(list(zip(src_first_test, src_second_test, tgt_first_test, tgt_second_test, all_first_test, all_second_test, 
                                         wei_first_test, wei_second_test, pre_first_test, pre_second_test, lin_first_test, lin_second_test)), columns=['SRCONLY_REG', 
                                                                                                                  'SRCONLY_NN', 'TGTONLY REG',
                                                                                                                  'TGTONLY NN', 'ALL_REG',
                                                                                                                  'ALL_NN', 'WEIGHTED_REG',
                                                                                                                  'WEIGHTED_NN', 'PRED_REG',
                                                                                                                  'PRED_NN', 'LININT_REG',
                                                                                                                  'LININT_NN'])

baseline_test_mse_select = pd.DataFrame(list(zip(all_first_test, all_second_test, wei_first_test, wei_second_test)), columns=['ALL_REG',
                                                                                                                  'ALL_NN', 'WEIGHTED_REG',
                                                                                                                  'WEIGHTED_NN'])


pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
baseline_test_mse.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,SRCONLY_REG,SRCONLY_NN,TGTONLY REG,TGTONLY NN,ALL_REG,ALL_NN,WEIGHTED_REG,WEIGHTED_NN,PRED_REG,PRED_NN,LININT_REG,LININT_NN
0,97.827539,99.073454,132.911066,121.071410,97.500958,92.235864,101.356653,107.257320,132.911066,125.094767,98.293617,91.394449
1,128.922144,134.576572,126.746146,130.107214,128.544959,125.682735,117.561373,121.054568,126.494276,133.024280,128.534463,124.018531
2,105.009285,105.741108,106.810751,102.258100,104.767162,101.469352,101.035803,123.463158,106.810751,109.606300,100.090359,102.117941


In [0]:
print(mean(all_first_test), mean(all_second_test), mean(wei_first_test), mean(wei_second_test))
baseline_test_mse_select.style.apply(highlight_min, axis=1)

110.27102637602123 106.46265061491978 106.65127642004931 117.2583486452573


,ALL_REG,ALL_NN,WEIGHTED_REG,WEIGHTED_NN
0,97.500958,92.235864,101.356653,107.257320
1,128.544959,125.682735,117.561373,121.054568
2,104.767162,101.469352,101.035803,123.463158


### Task 1.2 (Implement FEDA)

In [47]:
h,w = male_train.shape
zero_male = pd.DataFrame(0, index=range(h), columns=range(w))
male_train_feda = pd.concat([male_train, male_train, zero_male, zero_male], axis = 1, ignore_index=True)
print(male_train_feda.shape)
h,w = male_dev.shape
zero_male = pd.DataFrame(0, index=range(h), columns=range(w))
male_dev_feda = pd.concat([male_dev, male_dev, zero_male, zero_male], axis = 1, ignore_index=True)
h,w = male_test.shape
zero_male = pd.DataFrame(0, index=range(h), columns=range(w))
male_test_feda = pd.concat([male_test, male_test, zero_male, zero_male], axis = 1, ignore_index=True)
h,w = male_tgt.shape
zero_male = pd.DataFrame(0, index=range(h), columns=range(w))
male_tgt_feda = pd.concat([male_tgt, male_tgt, zero_male, zero_male], axis = 1, ignore_index=True)

h, w = female_train.shape
zero_female = pd.DataFrame(0, index=range(h), columns=range(w))
female_train_feda = pd.concat([female_train, zero_female, female_train, zero_female], axis = 1, ignore_index=True)
print(female_train_feda.shape)
h, w = female_dev.shape
zero_female = pd.DataFrame(0, index=range(h), columns=range(w))
female_dev_feda = pd.concat([female_dev, zero_female, female_dev, zero_female], axis = 1, ignore_index=True)
h, w = female_test.shape
zero_female = pd.DataFrame(0, index=range(h), columns=range(w))
female_test_feda = pd.concat([female_test, zero_female, female_test, zero_female], axis = 1, ignore_index=True)
h, w = female_tgt.shape
zero_female = pd.DataFrame(0, index=range(h), columns=range(w))
female_tgt_feda = pd.concat([female_tgt, zero_female, female_tgt, zero_female], axis = 1, ignore_index=True)

h, w = mixed_train.shape
zero_mixed = pd.DataFrame(0, index=range(h), columns=range(w))
mixed_train_feda = pd.concat([mixed_train, zero_mixed, zero_mixed, mixed_train], axis = 1, ignore_index=True)
print(mixed_train_feda.shape)
h, w = mixed_dev.shape
zero_mixed = pd.DataFrame(0, index=range(h), columns=range(w))
mixed_dev_feda = pd.concat([mixed_dev, zero_mixed, zero_mixed, mixed_dev], axis = 1, ignore_index=True)
h, w = mixed_test.shape
zero_mixed = pd.DataFrame(0, index=range(h), columns=range(w))
mixed_test_feda = pd.concat([mixed_test, zero_mixed, zero_mixed, mixed_test], axis = 1, ignore_index=True)
h, w = mixed_tgt.shape
zero_mixed = pd.DataFrame(0, index=range(h), columns=range(w))
mixed_tgt_feda = pd.concat([mixed_tgt, zero_mixed, zero_mixed, mixed_tgt], axis = 1, ignore_index=True)

#print(mixed_feda.tail(20))

(3454, 88)
(4204, 88)
(7089, 88)


In [48]:
print(male_tgt_feda.shape)

(100, 88)


### ALL

In [49]:
#100
#{'activation': 'relu', 'alpha': 0.01, 'batch_size': 200, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 1, 'batch_size': 200, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 1, 'batch_size': 64, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}

#300
#{'activation': 'relu', 'alpha': 1, 'batch_size': 200, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 1, 'batch_size': 64, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 200, 'hidden_layer_sizes': (30, 30, 30), 'learning_rate_init': 0.001, 'max_iter': 1000}

#500
#{'activation': 'relu', 'alpha': 0.01, 'batch_size': 200, 'hidden_layer_sizes': (30, 30, 30), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 1, 'batch_size': 200, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 1, 'batch_size': 128, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001, 'max_iter': 1000}

#700
#{'activation': 'relu', 'alpha': 1, 'batch_size': 128, 'hidden_layer_sizes': (30, 30, 30), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.01, 'batch_size': 200, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 128, 'hidden_layer_sizes': (30, 30, 30), 'learning_rate_init': 0.001, 'max_iter': 1000}

#900
#{'activation': 'relu', 'alpha': 0.01, 'batch_size': 200, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.01, 'batch_size': 200, 'hidden_layer_sizes': (30, 30, 30), 'learning_rate_init': 0.001, 'max_iter': 1000}
#{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 200, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'max_iter': 1000}

ma_train = pd.concat([female_train_feda, mixed_train_feda, male_tgt_feda], ignore_index=True)
ma_train = ma_train.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev = male_dev_feda.copy().reset_index(drop=True)
ma_test = male_test_feda.copy().reset_index(drop=True)
ma_train_pred = pd.concat([female_train_pred, mixed_train_pred, male_tgt_pred], ignore_index=True)
ma_train_pred = ma_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev_pred = male_dev_pred.copy().reset_index(drop=True)
ma_test_pred = male_test_pred.copy().reset_index(drop=True)

fe_train = pd.concat([male_train_feda, mixed_train_feda, female_tgt_feda], ignore_index=True)
fe_train = fe_train.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev = female_dev_feda.copy().reset_index(drop=True)
fe_test = female_test_feda.copy().reset_index(drop=True)
fe_train_pred = pd.concat([male_train_pred, mixed_train_pred, female_tgt_pred], ignore_index=True)
fe_train_pred = fe_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev_pred = female_dev_pred.copy().reset_index(drop=True)
fe_test_pred = female_test_pred.copy().reset_index(drop=True)

mi_train = pd.concat([male_train_feda, female_train_feda, mixed_tgt_feda], ignore_index=True)
mi_train = mi_train.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev = mixed_dev_feda.copy().reset_index(drop=True)
mi_test = mixed_test_feda.copy().reset_index(drop=True)
mi_train_pred = pd.concat([male_train_pred, female_train_pred, mixed_tgt_pred], ignore_index=True)
mi_train_pred = mi_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev_pred = mixed_dev_pred.copy().reset_index(drop=True)
mi_test_pred = mixed_test_pred.copy().reset_index(drop=True)

d = {'male':[ma_train, ma_dev, ma_test, ma_train_pred, ma_dev_pred, ma_test_pred], 'female':[fe_train, fe_dev, fe_test, fe_train_pred, fe_dev_pred, fe_test_pred], 'mixed':[mi_train, mi_dev, mi_test, mi_train_pred, mi_dev_pred, mi_test_pred]}

allfeda_first_dev_100 = []
allfeda_second_dev_100 = []
allfeda_first_test_100 = []
allfeda_second_test_100 = []

for item in d:
  df = d[item]
  X_train = df[0]
  Y_train = df[3]
  X_dev = df[1]
  Y_dev = df[4]
  X_test = df[2]
  Y_test = df[5]
  #scaler = StandardScaler()
  #scaler.fit(X_train)
  #X_train = scaler.transform(X_train)
  #X_dev = scaler.transform(X_dev)
  #X_test = scaler.transform(X_test)
  #parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
  #xgb = XGBClassifier(n_estimators=30, max_depth=10, random_state=24)
  #xgb.fit(X_train, Y_train)
  linr = LinearRegression() #normalize=True
  linr.fit(X_train, Y_train)
  allfeda_first_dev_100.append(mean_squared_error(Y_dev, linr.predict(X_dev)))
  allfeda_first_test_100.append(mean_squared_error(Y_test, linr.predict(X_test)))
  
  mlp = MLPRegressor(activation='relu', alpha=1, batch_size=200, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000, random_state=24, verbose=True)
  mlp.fit(X_train, Y_train)
  #mlp = MLPRegressor(random_state=24)
  #parameters = {'hidden_layer_sizes':[(100,), (50,50), (30,30,30), (25,25,25,25)], 'activation':['relu', 'identity'], 'alpha':[0.0001, 0.01, 0.1, 1, 10], 'batch_size':[200, 128, 64, 32], 'max_iter':[1000], 'learning_rate_init':[0.001, 0.01, 0.1]}
  #parameters = {'hidden_layer_sizes':[(100,), (50,50), (30,30,30)], 'alpha':[0.0001, 0.01, 1], 'activation':['relu'], 'max_iter':[1000], 'batch_size':[200, 128, 64], 'learning_rate_init':[0.001, 0.01, 0.1]}
  #parameters = {'hidden_layer_sizes':[(100,)], 'alpha':[0.0001], 'activation':['relu'], 'max_iter':[1000], 'batch_size':[200], 'learning_rate_init':[0.001, 0.01]}
  #mse = make_scorer(mean_squared_error, greater_is_better=False) #greater_is_better=False
  #clf = GridSearchCV(mlp, parameters, scoring = 'neg_mean_squared_error', n_jobs = -1)#, verbose=True, scoring = mse)
  #clf.fit(X_train, Y_train)
  #print(clf.best_params_)
  allfeda_second_dev_100.append(mean_squared_error(Y_dev, mlp.predict(X_dev)))
  allfeda_second_test_100.append(mean_squared_error(Y_test, mlp.predict(X_test)))

Iteration 1, loss = 123.43598851
Iteration 2, loss = 75.20827369
Iteration 3, loss = 70.70477638
Iteration 4, loss = 67.51675552
Iteration 5, loss = 64.65672063
Iteration 6, loss = 62.09307308
Iteration 7, loss = 59.82130659
Iteration 8, loss = 58.10812041
Iteration 9, loss = 56.70224104
Iteration 10, loss = 55.71267155
Iteration 11, loss = 55.14554993
Iteration 12, loss = 54.75312764
Iteration 13, loss = 54.30578141
Iteration 14, loss = 54.25111689
Iteration 15, loss = 54.24012464
Iteration 16, loss = 53.69347736
Iteration 17, loss = 53.48311526
Iteration 18, loss = 53.64370570
Iteration 19, loss = 53.14612845
Iteration 20, loss = 52.94441937
Iteration 21, loss = 53.07063468
Iteration 22, loss = 52.90928687
Iteration 23, loss = 52.76782654
Iteration 24, loss = 52.84212375
Iteration 25, loss = 52.85212785
Iteration 26, loss = 52.54980528
Iteration 27, loss = 52.28706860
Iteration 28, loss = 52.38476407
Iteration 29, loss = 52.22852872
Iteration 30, loss = 52.11088839
Iteration 31, loss

In [50]:
print(allfeda_first_test_100)
print(allfeda_second_test_100)

[132.90719813156554, 126.67705614703584, 106.8107507383397]
[106.52746594056809, 151.18696665059076, 101.79226924446758]


In [51]:
allfeda_test_mse_100 = pd.DataFrame(list(zip(allfeda_first_test_100, allfeda_second_test_100)), columns=['ALL_FEDA_REG','ALL_FEDA_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
allfeda_test_mse_100.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,ALL_FEDA_REG,ALL_FEDA_NN
0,132.907198,106.527466
1,126.677056,151.186967
2,106.810751,101.792269


In [44]:
print(allfeda_first_test_300)
print(allfeda_second_test_300)

[94.32761900374565, 118.369102871395, 97.05846931215302]
[93.07226816662556, 106.99407925986675, 94.87027837361717]


In [45]:
allfeda_test_mse_300 = pd.DataFrame(list(zip(allfeda_first_test_300, allfeda_second_test_300)), columns=['ALL_FEDA_REG','ALL_FEDA_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
allfeda_test_mse_300.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,ALL_FEDA_REG,ALL_FEDA_NN
0,94.327619,93.072268
1,118.369103,106.994079
2,97.058469,94.870278


In [38]:
print(allfeda_first_test_500)
print(allfeda_second_test_500)

[92.3379614788007, 115.8230226751933, 95.76670896552042]
[93.7073724409872, 112.31518351953665, 93.65537818401909]


In [39]:
allfeda_test_mse_500 = pd.DataFrame(list(zip(allfeda_first_test_500, allfeda_second_test_500)), columns=['ALL_FEDA_REG','ALL_FEDA_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
allfeda_test_mse_500.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,ALL_FEDA_REG,ALL_FEDA_NN
0,92.337961,93.707372
1,115.823023,112.315184
2,95.766709,93.655378


In [32]:
print(allfeda_first_test_700)
print(allfeda_second_test_700)

[93.12344443406728, 115.6184885666003, 95.92172751926219]
[92.10129437233205, 100.94481316476417, 89.39373372959298]


In [33]:
allfeda_test_mse_700 = pd.DataFrame(list(zip(allfeda_first_test_700, allfeda_second_test_700)), columns=['ALL_FEDA_REG','ALL_FEDA_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
allfeda_test_mse_700.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,ALL_FEDA_REG,ALL_FEDA_NN
0,93.123444,92.101294
1,115.618489,100.944813
2,95.921728,89.393734


In [26]:
print(allfeda_first_test_900)
print(allfeda_second_test_900)

[91.0174345515029, 116.53247485433006, 95.77382759053945]
[88.71900276088115, 105.62101362817866, 93.54721656650149]


In [27]:
allfeda_test_mse_900 = pd.DataFrame(list(zip(allfeda_first_test_900, allfeda_second_test_900)), columns=['ALL_FEDA_REG','ALL_FEDA_NN'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
allfeda_test_mse_900.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,ALL_FEDA_REG,ALL_FEDA_NN
0,91.017435,88.719003
1,116.532475,105.621014
2,95.773828,93.547217


### Task 2 (Implement your own model)

In [0]:
ma_train = pd.concat([female_train, mixed_train, male_tgt], ignore_index=True)
#ma_train = ma_train.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev = male_dev.copy().reset_index(drop=True)
ma_test = male_test.copy().reset_index(drop=True)
ma_train_pred = pd.concat([female_train_pred, mixed_train_pred, male_tgt_pred], ignore_index=True)
#ma_train_pred = ma_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
ma_dev_pred = male_dev_pred.copy().reset_index(drop=True)
ma_test_pred = male_test_pred.copy().reset_index(drop=True)

fe_train = pd.concat([male_train, mixed_train, female_tgt], ignore_index=True)
#fe_train = fe_train.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev = female_dev.copy().reset_index(drop=True)
fe_test = female_test.copy().reset_index(drop=True)
fe_train_pred = pd.concat([male_train_pred, mixed_train_pred, female_tgt_pred], ignore_index=True)
#fe_train_pred = fe_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
fe_dev_pred = female_dev_pred.copy().reset_index(drop=True)
fe_test_pred = female_test_pred.copy().reset_index(drop=True)

mi_train = pd.concat([male_train, female_train, mixed_tgt], ignore_index=True)
#mi_train = mi_train.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev = mixed_dev.copy().reset_index(drop=True)
mi_test = mixed_test.copy().reset_index(drop=True)
mi_train_pred = pd.concat([male_train_pred, female_train_pred, mixed_tgt_pred], ignore_index=True)
#mi_train_pred = mi_train_pred.sample(frac=1, random_state = 24).reset_index(drop=True)
mi_dev_pred = mixed_dev_pred.copy().reset_index(drop=True)
mi_test_pred = mixed_test_pred.copy().reset_index(drop=True)

d = {'male':[ma_train, ma_dev, ma_test, ma_train_pred, ma_dev_pred, ma_test_pred], 'female':[fe_train, fe_dev, fe_test, fe_train_pred, fe_dev_pred, fe_test_pred], 'mixed':[mi_train, mi_dev, mi_test, mi_train_pred, mi_dev_pred, mi_test_pred]}

tr_first_dev = []
tr_second_dev = []
tr_first_test = []
tr_second_test = []

for item in d:
  df = d[item]
  X_train = df[0]
  Y_train = df[3]
  X_dev = df[1]
  Y_dev = df[4]
  X_test = df[2]
  Y_test = df[5]
  w = np.ones(len(Y_train))#/len(Y_train)
  #print(X_train.shape)
  n = X_train.shape[0] - 100
  #print(n)
  #print(w.shape)
  N = 3
  for i in range(N):
    denom = np.sum(w)
    p = np.divide(w, denom)
    #X_train_new = X_train.copy()
    #X_train_new['Prob'] = p
    linr = LinearRegression()
    linr.fit(X_train, Y_train, sample_weight=p)
    x_s = X_train.tail(100)
    x_t = X_train.head(n)
    c_s = Y_train.tail(100).values
    #print(c_s.shape)
    c_t = Y_train.head(n).values
    w_m = w[-100:]
    den = np.sum(w_m)
    #print(x.shape, c.shape, len(w_m))
    h_s = linr.predict(x_s)
    h_t = linr.predict(x_t)
    diff = np.absolute(h_s-c_s)
    num = np.dot(w_m, diff)
    e = np.divide(num,den)
    if e >= 0.5:
      e = 0.5
    if e == 0:
      break 
    bt = np.divide(e, 1-e)
    b = 1 / (1 + np.sqrt(2 * np.log(n) / N))
    for j in range(n):
      #print(j)
      w[j] = w[j] * np.power(b, np.absolute(h_t[j] - c_t[j]))
    for j in range(100):
      #print(j)
      #print(w[n+j])
      #print(h_s[j])
      #print(c_s[j])
      w[n + j] = w[n + j] * np.power(bt,(-np.absolute(h_s[j] - c_s[j])))
  print(w)
  tr_first_dev.append(mean_squared_error(Y_dev, linr.predict(X_dev)))
  tr_first_test.append(mean_squared_error(Y_test, linr.predict(X_test)))

  w = np.ones(len(Y_train))#/len(Y_train)
  #print(X_train.shape)
  n = X_train.shape[0] - 100
  #print(n)
  #print(w.shape)
  N = 50
  for i in range(N):
    denom = np.sum(w)
    p = np.divide(w, denom)
    #X_train_new = X_train.copy()
    #X_train_new['Prob'] = p
    #clf = XGBClassifier(n_estimators=30, max_depth=10, random_state=24)
    clf = DecisionTreeClassifier(criterion="gini", max_features="log2", splitter="random")
    #clf = LogisticRegression(class_weight="balanced", max_iter=1000, random_state=24)
    #mlp = MLPRegressor(activation='relu', alpha=0.0001, batch_size=64, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000, random_state=24, verbose=True)
    clf.fit(X_train, Y_train, sample_weight=p)
    x_s = X_train.tail(100)
    x_t = X_train.head(n)
    c_s = Y_train.tail(100).values
    #print(c_s.shape)
    c_t = Y_train.head(n).values
    w_m = w[-100:]
    den = np.sum(w_m)
    #print(x.shape, c.shape, len(w_m))
    h_s = linr.predict(x_s)
    h_t = linr.predict(x_t)
    diff = np.absolute(h_s-c_s)
    num = np.dot(w_m, diff)
    e = np.divide(num,den)
    if e > 0.5:
      e = 0.5
    if e == 0:
      break 
    bt = np.divide(e, 1-e)
    b = 1 / (1 + np.sqrt(2 * np.log(n) / N))
    for j in range(n):
      #print(j)
      w[j] = w[j] * np.power(b, np.absolute(h_t[j] - c_t[j]))
    for j in range(100):
      #print(j)
      #print(w[n+j])
      #print(h_s[j])
      #print(c_s[j])
      w[n + j] = w[n + j] * np.power(bt,(-np.absolute(h_s[j] - c_s[j])))
  print(w)
  tr_second_dev.append(mean_squared_error(Y_dev, clf.predict(X_dev)))
  tr_second_test.append(mean_squared_error(Y_test, clf.predict(X_test)))

[3.24913785e-21 5.28093887e-11 4.44134605e-03 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
[2.35077587e-125 5.49553952e-070 9.88772883e-023 ... 1.00000000e+000
 1.00000000e+000 1.00000000e+000]
[6.87229162e-13 3.72214134e-23 2.00260435e-07 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
[1.37017881e-092 1.25479529e-142 5.24619778e-038 ... 1.00000000e+000
 1.00000000e+000 1.00000000e+000]
[1.18267965e-11 1.11740106e-20 2.36174839e-08 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
[7.55011827e-067 1.93171782e-131 3.56147722e-044 ... 1.00000000e+000
 1.00000000e+000 1.00000000e+000]


In [0]:
print(tr_first_dev)
print(tr_first_test)
print(tr_second_dev)
print(tr_second_test)

[136.33996635508927, 124.77459261773818, 109.17517637844475]
[101.48663209258373, 118.46322402014235, 101.96992003092492]
[146.77, 120.89, 168.12]
[114.42, 147.43, 121.94]


In [0]:
tr_test_mse = pd.DataFrame(list(zip(tr_first_test, tr_second_test)), columns=['TrADABOOST_REG','TrADABOOST_DT'])
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_columns', None)
tr_test_mse.style.apply(highlight_min, axis=1)
#print(baseline_dev_mse.head())

,TrADABOOST_REG,TrADABOOST_DT
0,101.486632,114.420000
1,118.463224,147.430000
2,101.969920,121.940000
